<a href="https://colab.research.google.com/github/YannickPezeu/Titanic-Competition/blob/main/Tensorflow_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input
from sklearn.model_selection import train_test_split
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import cross_val_score


#data = pd.read_csv("gs://titanic_yannick_pezeu/titanic/Preprocessed Data/train_preprocessed.csv")
data = pd.read_csv("gs://titanic_yannick_pezeu/titanic/Preprocessed Data/train_preprocessed_tensorflow.csv")

data["Cabin"].replace(np.nan, "0", inplace = True) 
data

,PassengerId,Survived,Pclass,Title,LastName,Sex,Age,SibSp,Parch,Fare,Cabin,CabinF,CabinA,CabinB,CabinC,CabinE,CabinD,Embarked
0,1,0,3,Mr,Braund,male,22,1,0,7.2500,0,0,0,0,0,0,0,S
1,2,1,1,Mrs,Cumings,female,38,1,0,71.2833,C85,0,0,0,1,0,0,C
2,3,1,3,Miss,Heikkinen,female,26,0,0,7.9250,0,0,0,0,0,0,0,S
3,4,1,1,Mrs,Futrelle,female,35,1,0,53.1000,C123,0,0,0,1,0,0,S
4,5,0,3,Mr,Allen,male,35,0,0,8.0500,0,0,0,0,0,0,0,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,Rev,Montvila,male,27,0,0,13.0000,0,0,0,0,0,0,0,S
887,888,1,1,Miss,Graham,female,19,0,0,30.0000,B42,0,0,1,0,0,0,S
888,889,0,3,Miss,Johnston,female,30,1,2,23.4500,0,0,0,0,0,0,0,S
889,890,1,1,Mr,Behr,male,26,0,0,30.0000,C148,0,0,0,1,0,0,C


In [ ]:
data["Embarked"].replace(np.nan, "0", inplace = True) 

# Prepare Dataset

In [ ]:
train, test = train_test_split(data, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

569 train examples
143 validation examples
179 test examples


In [ ]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Survived')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [ ]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [ ]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['Age'])
  print('A batch of targets:', label_batch )

Every feature: ['PassengerId', 'Pclass', 'Title', 'LastName', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'CabinF', 'CabinA', 'CabinB', 'CabinC', 'CabinE', 'CabinD', 'Embarked']
A batch of ages: tf.Tensor([22 36 23 54 33], shape=(5,), dtype=int64)
A batch of targets: tf.Tensor([1 0 1 0 0], shape=(5,), dtype=int64)


In [ ]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

In [ ]:
# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [ ]:
photo_count = feature_column.numeric_column('Age')
demo(photo_count)

[[32.]
 [30.]
 [23.]
 [30.]
 [41.]]


In [ ]:
age = feature_column.numeric_column('Age')
age_buckets = feature_column.bucketized_column(age, boundaries=[1, 20, 50])
demo(age_buckets)

[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]]


In [ ]:
Cabine = feature_column.categorical_column_with_hash_bucket("Cabin", 1000)

Cabine = feature_column.indicator_column(Cabine)
demo(Cabine)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
Sex = feature_column.categorical_column_with_vocabulary_list("Sex", ["male", "female"])

Sex = feature_column.indicator_column(Sex)
demo(Sex)

[[0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [ ]:
list(data["Pclass"].unique())

[3, 1, 2]

In [ ]:
data

,PassengerId,Survived,Pclass,Title,LastName,Sex,Age,SibSp,Parch,Fare,Cabin,CabinF,CabinA,CabinB,CabinC,CabinE,CabinD,Embarked
0,1,0,3,Mr,Braund,male,22,1,0,7.2500,0,0,0,0,0,0,0,S
1,2,1,1,Mrs,Cumings,female,38,1,0,71.2833,C85,0,0,0,1,0,0,C
2,3,1,3,Miss,Heikkinen,female,26,0,0,7.9250,0,0,0,0,0,0,0,S
3,4,1,1,Mrs,Futrelle,female,35,1,0,53.1000,C123,0,0,0,1,0,0,S
4,5,0,3,Mr,Allen,male,35,0,0,8.0500,0,0,0,0,0,0,0,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,Rev,Montvila,male,27,0,0,13.0000,0,0,0,0,0,0,0,S
887,888,1,1,Miss,Graham,female,19,0,0,30.0000,B42,0,0,1,0,0,0,S
888,889,0,3,Miss,Johnston,female,30,1,2,23.4500,0,0,0,0,0,0,0,S
889,890,1,1,Mr,Behr,male,26,0,0,30.0000,C148,0,0,0,1,0,0,C


# Sequential Model

In [ ]:
categorical_columns_vocabulary_names = ["Pclass", "Title", "Sex", "Embarked", "SibSp", "Parch"]
categorical_columns_hash_names = ["Cabin", "LastName"]
numeric_columns_names = ["Age", "Fare"]
indicator_columns_names = ["CabinA","CabinB","CabinC","CabinD","CabinE","CabinF"]

column_names = categorical_columns_vocabulary_names+categorical_columns_hash_names +numeric_columns_names + indicator_columns_names


In [ ]:
feature_columns = []

# numeric cols
for header in numeric_columns_names:
  feature_columns.append(feature_column.numeric_column(header))

# bucketized cols
age = feature_column.numeric_column('Age')
age_buckets = feature_column.bucketized_column(age, boundaries=[0, 5, 10, 15, 20,25,30,35,40,45,50,55,60,65,70,75,80])
feature_columns.append(age_buckets)

# indicator_columns
for col_name in indicator_columns_names:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, data[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  feature_columns.append(indicator_column)

# categorical with vocabulary
for col_name in categorical_columns_vocabulary_names:
    categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, data[col_name].unique())
    indicator_column = feature_column.indicator_column(categorical_column)
    feature_columns.append(indicator_column)
    
for col_name in categorical_columns_hash_names:
    categorical_column = feature_column.categorical_column_with_hash_bucket(
      col_name, 1000)
    indicator_column = feature_column.indicator_column(categorical_column)
    feature_columns.append(indicator_column)


    

In [ ]:
feature_columns

[NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Fare', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 BucketizedColumn(source_column=NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='CabinA', vocabulary_list=(0, 1), dtype=tf.int64, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='CabinB', vocabulary_list=(0, 1), dtype=tf.int64, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='CabinC', vocabulary_list=(0, 1), dtype=tf.int64, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='CabinD', vocabulary_list=(0, 1), dtype=tf

In [ ]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [ ]:
feature_layer(inputs)

NameError: name 'inputs' is not defined

In [ ]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [ ]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.3),
  layers.Dense(1, activation = tf.keras.activations.sigmoid)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=12)

Epoch 1/12
Consider rewriting this model with the Functional API.


/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/backend.py:5017: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


Consider rewriting this model with the Functional API.
17/18 [===========================>..] - ETA: 0s - loss: 0.7080 - accuracy: 0.6360WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'PassengerId': <tf.Tensor 'ExpandDims_12:0' shape=(None, 1) dtype=int64>, 'Pclass': <tf.Tensor 'ExpandDims_13:0' shape=(None, 1) dtype=int64>, 'Title': <tf.Tensor 'ExpandDims_16:0' shape=(None, 1) dtype=string>, 'LastName': <tf.Tensor 'ExpandDims_10:0' shape=(None, 1) dtype=string>, 'Sex': <tf.Tensor 'ExpandDims_14:0' shape=(None, 1) dtype=string>, 'Age': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=int64>, 'SibSp': <tf.Tensor 'ExpandDims_15:0' shape=(None, 1) dtype=int64>, 'Parch': <tf.Tensor 'ExpandDims_11:0' shape=(None, 1) dtype=int64>, 'Fare': <tf.Tensor 'ExpandDims_9:0' shape=(None, 1) dtype=float64>, 'Cabin': <tf.Tensor 'ExpandDims_1:0' shape=(None, 1) dtype=string>, 'CabinF': <tf.Tensor 'ExpandDims_7:0' shape=(None

# Functional Model

In [ ]:
categorical_columns_vocabulary_names = ["Pclass", "Title", "Sex", "Embarked", "SibSp", "Parch"]
categorical_columns_hash_names = ["Cabin", "LastName"]
numeric_columns_names = ["Age", "Fare"]
indicator_columns_names = ["CabinA","CabinB","CabinC","CabinD","CabinE","CabinF"]

column_names = categorical_columns_vocabulary_names+categorical_columns_hash_names +numeric_columns_names + indicator_columns_names

deep_columns = []
for header in numeric_columns_names:
  deep_columns.append(feature_column.numeric_column(header))

wide_columns = []
for col_name in categorical_columns_vocabulary_names:
    categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, data[col_name].unique())
    indicator_column = feature_column.indicator_column(categorical_column)
    wide_columns.append(indicator_column)
    
for col_name in categorical_columns_hash_names:
    categorical_column = feature_column.categorical_column_with_hash_bucket(
      col_name, 1000)
    indicator_column = feature_column.indicator_column(categorical_column)
    wide_columns.append(indicator_column)

# bucketized cols
age = feature_column.numeric_column('Age')
age_buckets = feature_column.bucketized_column(age, boundaries=[0, 5, 10, 15, 20,25,30,35,40,45,50,55,60,65,70,75,80])
wide_columns.append(age_buckets)

# indicator_columns
for col_name in indicator_columns_names:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, data[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  wide_columns.append(indicator_column)    

inputs = {colname: Input(name = colname, shape = (), dtype=data[colname].dtype) for colname in column_names}
deep = tf.keras.layers.DenseFeatures(deep_columns)(inputs)
wide = tf.keras.layers.DenseFeatures(wide_columns)(inputs)

In [ ]:
deep = tf.keras.layers.Dense(128, activation = 'relu')(deep)
deep = layers.Dropout(.3)(deep)
deep = tf.keras.layers.Dense(12, activation = 'relu')(deep)
deep = layers.Dropout(.3)(deep)
mix = tf.keras.layers.concatenate([deep, wide])
mix = tf.keras.layers.Dense(12, activation = 'relu')(mix)
mix = layers.Dropout(.3)(mix)
output = tf.keras.layers.Dense(1, activation = tf.keras.activations.sigmoid)(mix)

In [ ]:
model_functional = tf.keras.Model(inputs = inputs, outputs = output, name = 'model_functional')

In [ ]:
model_functional.summary()

Model: "model_functional"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Age (InputLayer)                [(None,)]            0                                            
__________________________________________________________________________________________________
Cabin (InputLayer)              [(None,)]            0                                            
__________________________________________________________________________________________________
CabinA (InputLayer)             [(None,)]            0                                            
__________________________________________________________________________________________________
CabinB (InputLayer)             [(None,)]            0                                            
___________________________________________________________________________________

In [ ]:
model_functional.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_functional.fit(train_ds,
          validation_data=val_ds,
          epochs=30)

Epoch 1/30


/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/functional.py:591: UserWarning: Input dict contained keys ['PassengerId'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


18/18 [==============================] - 6s 104ms/step - loss: 0.7105 - accuracy: 0.5466 - val_loss: 0.6497 - val_accuracy: 0.6643
Epoch 2/30
18/18 [==============================] - 1s 30ms/step - loss: 0.6463 - accuracy: 0.6749 - val_loss: 0.6191 - val_accuracy: 0.7133
Epoch 3/30
18/18 [==============================] - 1s 37ms/step - loss: 0.6298 - accuracy: 0.6714 - val_loss: 0.5831 - val_accuracy: 0.6993
Epoch 4/30
18/18 [==============================] - 1s 49ms/step - loss: 0.6087 - accuracy: 0.6837 - val_loss: 0.5689 - val_accuracy: 0.7413
Epoch 5/30
18/18 [==============================] - 1s 42ms/step - loss: 0.5769 - accuracy: 0.6924 - val_loss: 0.5446 - val_accuracy: 0.7622
Epoch 6/30
18/18 [==============================] - 1s 65ms/step - loss: 0.5607 - accuracy: 0.7118 - val_loss: 0.5245 - val_accuracy: 0.7692
Epoch 7/30
18/18 [==============================] - 1s 45ms/step - loss: 0.5508 - accuracy: 0.7381 - val_loss: 0.5142 - val_accuracy: 0.7832
Epoch 8/30
18/18 [=====

# Second Functional Model

In [ ]:
deep = tf.keras.layers.DenseFeatures(deep_columns)(inputs)
wide = tf.keras.layers.DenseFeatures(wide_columns)(inputs)

deep = tf.keras.layers.Dense(128, activation = 'relu',  kernel_regularizer=tf.keras.regularizers.L1(0.01))(deep)
deep = layers.Dropout(.3)(deep)
deep = tf.keras.layers.Dense(20, activation = 'relu')(deep)
deep = layers.Dropout(.3)(deep)
mix = tf.keras.layers.concatenate([deep, wide])
mix = tf.keras.layers.Dense(12, activation = 'relu')(mix)
mix = layers.Dropout(.3)(mix)
output = tf.keras.layers.Dense(1, activation = tf.keras.activations.sigmoid)(mix)

model_functional = tf.keras.Model(inputs = inputs, outputs = output, name = 'model_functional')

model_functional.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_functional.fit(train_ds,
          validation_data=val_ds,
          epochs=40)

# Third Functional Model

In [ ]:
deep = tf.keras.layers.DenseFeatures(deep_columns)(inputs)
wide = tf.keras.layers.DenseFeatures(wide_columns)(inputs)

deep = tf.keras.layers.Dense(128, activation = 'relu',  kernel_regularizer=tf.keras.regularizers.L1(0.01))(deep)
deep = layers.Dropout(.3)(deep)
deep = tf.keras.layers.Dense(20, activation = 'relu')(deep)
deep = layers.Dropout(.3)(deep)
mix = tf.keras.layers.concatenate([deep, wide])
mix = tf.keras.layers.Dense(20, activation = 'relu')(mix)
mix = layers.Dropout(.3)(mix)
output = tf.keras.layers.Dense(1, activation = tf.keras.activations.sigmoid)(mix)

model_functional = tf.keras.Model(inputs = inputs, outputs = output, name = 'model_functional')

model_functional.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_functional.fit(train_ds,
          validation_data=val_ds,
          epochs=40)

In [ ]:
model_functional.evaluate(val_ds)[1]

# Bayesian Optimisation


In [ ]:
from skopt.space import Integer
from skopt.space import Real
from skopt.utils import use_named_args
from skopt import gp_minimize

In [ ]:
def create_model(l1, l2, l3, dropout, regulariser_coeff, epochs):
    deep = tf.keras.layers.DenseFeatures(deep_columns)(inputs)
    wide = tf.keras.layers.DenseFeatures(wide_columns)(inputs)

    deep = tf.keras.layers.Dense(l1, activation = 'relu',  kernel_regularizer=tf.keras.regularizers.L1(regulariser_coeff))(deep)
    deep = layers.Dropout(dropout)(deep)
    deep = tf.keras.layers.Dense(l2, activation = 'relu')(deep)
    deep = layers.Dropout(dropout)(deep)
    mix = tf.keras.layers.concatenate([deep, wide])
    mix = tf.keras.layers.Dense(l3, activation = 'relu')(mix)
    mix = layers.Dropout(dropout)(mix)
    output = tf.keras.layers.Dense(1, activation = tf.keras.activations.sigmoid)(mix)

    model_functional = tf.keras.Model(inputs = inputs, outputs = output, name = 'model_functional')

    model_functional.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    model_functional.fit(train_ds,
              validation_data=val_ds,
              epochs=epochs)
    return model_functional

In [ ]:
def evaluate_model(model):
    evaluation = model.evaluate(val_ds)[1]
    return 1 - evaluation

In [ ]:
search_space = [Integer(20, 200, name='l1'),Integer(20, 200, name='l2'), Integer(20, 200, name='l3'), 
                Real(0, 0.5, name = "dropout"), Integer(10,30, name = "epochs"), Real(0,0.5, name = "regulariser_coeff")]

In [ ]:
@use_named_args(search_space)
def evaluate_params(l1, l2, l3, dropout, regulariser_coeff, epochs):
    print("l1: ", l1, "l2:", l2, "l3:", l3, "dropout:", dropout, "regulariser_coeff", regulariser_coeff, "epochs:", epochs )
    model = create_model(l1, l2, l3, dropout, regulariser_coeff, epochs)
    return evaluate_model(model)
    

In [ ]:
result = gp_minimize(evaluate_params, search_space)

l1:  135 l2: 101 l3: 53 dropout: 0.16483492199639968 regulariser_coeff 0.4393393858088332 epochs: 26
Epoch 1/26
18/18 [==============================] - 4s 88ms/step - loss: 12.3127 - accuracy: 0.6503 - val_loss: 11.1428 - val_accuracy: 0.7133
Epoch 2/26
18/18 [==============================] - 1s 42ms/step - loss: 10.2989 - accuracy: 0.6731 - val_loss: 9.2726 - val_accuracy: 0.7483
Epoch 3/26
18/18 [==============================] - 1s 30ms/step - loss: 8.5057 - accuracy: 0.6960 - val_loss: 7.5995 - val_accuracy: 0.7622
Epoch 4/26
18/18 [==============================] - 1s 39ms/step - loss: 6.8860 - accuracy: 0.7329 - val_loss: 6.0626 - val_accuracy: 0.7762
Epoch 5/26
18/18 [==============================] - 1s 30ms/step - loss: 5.4418 - accuracy: 0.7663 - val_loss: 4.7400 - val_accuracy: 0.7972
Epoch 6/26
18/18 [==============================] - 1s 44ms/step - loss: 4.1814 - accuracy: 0.8014 - val_loss: 3.5591 - val_accuracy: 0.8112
Epoch 7/26
18/18 [==============================] 